In [3]:
import os
import pandas as pd
import re
import json


class TextPreprocessor:
    def __init__(self):
        self.stop_phrases = [
            'Ещё больше новостей — в телеграм-канале Москва 24 Подписывайтесь!', 
            'Подробнее – в эфире телеканала Москва 24.'
        ]

        self.tags_keyword = '\nTags: '

        self.word_with_capital_letter_pattern = re.compile('^[А-ЯA-Z]')
        self.sentence_ending_char_on_new_line_pattern = re.compile('(?<=[а-я])\n(?=\\.)')
        self.footnote_pattern = re.compile('^\\*+')
        self.citation_beginning_pattern = re.compile('^("|“|«)')


    def preprocess_text(self, text: str):
        if self.tags_keyword in text:
            _text, tags_sequence = text.split(self.tags_keyword)

            tags = [t.strip(' ') for t in tags_sequence.split(',')]
        else:
            _text = text + ''

            tags = []

        lines = _text.split('\n')

        relevant_lines = []
        footnotes = []

        for l in lines:
            l = l.strip(' \t')

            for phi in self.stop_phrases:
                l = l.replace(phi, '')

            if len(l) == 0:
                continue

            if bool(self.word_with_capital_letter_pattern.search(l)) or bool(self.citation_beginning_pattern.search(l)):
                if len(relevant_lines) == 0:
                    relevant_lines.append(l)
                else:
                    relevant_lines[-1] += ' ' + l
            elif l.startswith('.') or l.startswith(','):
                if len(relevant_lines) == 0:
                    relevant_lines.append(l)
                else:
                    relevant_lines[-1] += l
            elif bool(self.footnote_pattern.search(l)):
                footnotes.append(l)
            else:
                relevant_lines.append(l)

        return relevant_lines, tags, footnotes

In [4]:
import json


root_path = 'datasets'

filename = 'test_assignment_data.csv'

raw_data = pd.read_csv(os.path.join(root_path, filename))

preprocessor = TextPreprocessor()

complete_texts_dataset = []
incomplete_texts_dataset = []

for i, row in raw_data.iterrows():
    text_lines, tags, footnotes = preprocessor.preprocess_text(row['fullText'])

    if len(text_lines) > 1:
        incomplete_texts_dataset.append({
                'fullText': '\n'.join(text_lines), 
                'pubTime': row['pubTime'], 
                'tags': tags, 
                'footnotes': footnotes
            })
    else:
        complete_texts_dataset.append({
                'fullText': '\n'.join(text_lines), 
                'pubTime': row['pubTime'], 
                'tags': tags, 
                'footnotes': footnotes
            })
    
# with open(os.path.join(root_path, 'processed_dataset.json'), 'w') as f:
#     json.dump(incomplete_texts_dataset + complete_texts_dataset, f)

# with open(os.path.join(root_path, 'complete_texts_dataset.json'), 'w') as f:
#     json.dump(complete_texts_dataset, f)

# with open(os.path.join(root_path, 'incomplete_texts_dataset.json'), 'w') as f:
#     json.dump(incomplete_texts_dataset, f)

In [5]:
for item in incomplete_texts_dataset[:15]:
    print(item['fullText'])
    print('=======================')

Три дома, в которых в свое время жили сотрудники трех разных ведомств, капитально отремонтировали на Ленинградском проспекте в 2023 году. Об этом сообщает портал мэра и правительства столицы со ссылкой на комплекс
ского хозяйства Москвы.Так, в рамках программы капремонта жилого фонда завершились работы по обновлению дома 65. Семиэтажное здание построили в 1935 году по индивидуальному проекту в стиле советского неоклассицизма. Отмечается, что первыми там квартиры получили сотрудники Государственного комитета СССР по телевидению и радиовещанию. В ходе капремонта специалисты привели в порядок фасад дома, отремонтировали подвал, балконы, плиты и напольное покрытие, заменили магистрали коммуникаций, обустроили цоколь, навесили новые трубы наружного водостока, заменили входные двери и установили современные оконные блоки.Также капитально отремонтирован дом 13, строение 1а. Здание построили в 1949 году по индивидуальному проекту. Изначально его возводили для сотрудников "Мосфильма", но к окон